<a href="https://colab.research.google.com/github/pgbio99/Study_Gene-Expression_-Chemoresistance-in-Cancer_Machine-learning/blob/main/Master_Union_Biomarkers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving significant_DEGs.csv to significant_DEGs (1).csv
Saving significant_DEGs_2.csv to significant_DEGs_2.csv
Saving significant_DEGs_b.csv to significant_DEGs_b.csv
Saving significant_DEGs_n.csv to significant_DEGs_n.csv


In [ ]:
DEA_FILE_PATHS = {
    'A' : 'significant_DEGs.csv',
    'B' : 'significant_DEGs_2.csv',
    'C' : 'significant_DEGs_b.csv',
    'D' : 'significant_DEGs_n.csv'
}
TOP_N = 100
Output_file = 'Master_Union_Biomarkers_N.csv'

In [ ]:
gene_sets = []

for study_name, file_path in DEA_FILE_PATHS.items():
    try:
        df = pd.read_csv(file_path)

        df['Abs_Log2FC'] = df['log2FC'].abs()

        current_gene_count = len(df)

        if current_gene_count > TOP_N:
            n_to_select = TOP_N
        else:
            # If the list is small (like 2 genes), take all of them
            n_to_select = current_gene_count


        top_genes_list = df.sort_values(
            'Abs_Log2FC',
            ascending=False
        ).head(n_to_select)['Gene'].tolist()

        gene_sets.append(set(top_genes_list))

        print(f"Study {study_name}: Selected {n_to_select} genes (Original count: {current_gene_count}).")

    except FileNotFoundError:
        print(f"Error: DEA file for Study {study_name} not found at {file_path}. Skipping.")
    except KeyError as e:
        print(f"Error in {study_name}: Required column not found. Check if 'Gene' or 'log2FC' exists.")

Study A: Selected 2 genes (Original count: 2).
Study B: Selected 100 genes (Original count: 751).
Study C: Selected 100 genes (Original count: 8250).
Study D: Selected 100 genes (Original count: 362).


In [ ]:
master_feature_set = set().union(*gene_sets)

print(f"\nTotal Unique Master Features for ML (Original Names): {len(master_feature_set)}")


Total Unique Master Features for ML (Original Names): 298


In [ ]:
master_df = pd.DataFrame(
    list(master_feature_set),
    columns=['Gene']
)
master_df.to_csv(Output_file, index=False, sep='\t')
print(f"✅ Master feature list with ORIGINAL GENE NAMES saved to {Output_file}")

✅ Master feature list with ORIGINAL GENE NAMES saved to Master_Union_Biomarkers_N.csv


In [ ]:
master_genes_df = pd.read_csv('Master_Union_Biomarkers_N.csv', sep='\t')
master_genes = set(master_genes_df['Gene'].tolist())

In [ ]:
GENE_ID_COL = 'Gene'
OUTPUT_FILENAME = 'Top_Biomarkers_Statistics_Annotated.csv'

In [ ]:
combined_results = []
for study, path in DEA_FILE_PATHS.items():
    try:
        df_dea = pd.read_csv(path)

        # Filter the DEA results to only include the Master Genes
        filtered_df = df_dea[df_dea[GENE_ID_COL].isin(master_genes)].copy()

        if filtered_df.empty:
            print(f"Warning: No master genes found in {study}'s DEA file. This might be correct if the study contributed few genes.")
            continue

        filtered_df['Source_Study'] = study
        filtered_df = filtered_df[[GENE_ID_COL, 'log2FC', 'FDR_P_VALUE', 'Source_Study']]

        filtered_df.rename(columns={GENE_ID_COL: 'Biological_Gene_ID'}, inplace=True)

        combined_results.append(filtered_df)

    except FileNotFoundError:
        print(f"Error: DEA file for Study {study} not found at {path}. Skipping.")
    except KeyError as e:
        print(f"Error: Required column not found in DEA file for Study {study}: {e}.")

In [ ]:
final_biomarker_stats = pd.concat(combined_results, ignore_index=True)
final_biomarker_stats = final_biomarker_stats.sort_values(
    ['Biological_Gene_ID', 'FDR_P_VALUE']
)
final_biomarker_stats.to_csv(OUTPUT_FILENAME, index=False)

print("\n--- Final Annotated Biomarkers File Created ---")
print(f"The list now contains the original gene identifiers in the 'Biological_Gene_ID' column and is ready for ML setup.")


--- Final Annotated Biomarkers File Created ---
The list now contains the original gene identifiers in the 'Biological_Gene_ID' column and is ready for ML setup.
